In [1]:
import pandas as pd
import numpy as np
import librosa
from librosa import feature
from glob import glob
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
rootdir = '../audio-files'
# rootdir="audio_files"
audio_files = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        audio_files.append(os.path.join(subdir, file))


In [3]:
len(audio_files)

1440

In [4]:
dict_actors = {}
for audio_file in audio_files:
    try :
        if (len(dict_actors[audio_file[-33:-25]])>0):
            dict_actors[audio_file[-33:-25]].append(audio_file)
    except:
        dict_actors[audio_file[-33:-25]] = []
        dict_actors[audio_file[-33:-25]].append(audio_file)

In [5]:
data = pd.DataFrame(columns = ['Actor','Modality','Vocal_channel','Emotion','Emotional_intensity','Statement','Repetion','Gender','Audio_file'])

In [6]:
def get_gender(value):
    if(int(value)%2==0):
        return 1
    else :
        return 0

for keys in dict_actors:
    for files in dict_actors[keys]:
        data.loc[len(data)] = [keys,int(files[-24:-22]),int(files[-21:-19]),int(files[-18:-16]),int(files[-15:-13]),int(files[-12:-10]),int(files[-9:-7]),get_gender(files[-6:-4]),files]

In [7]:
data.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender,Audio_file
0,Actor_01,3,1,1,1,1,1,0,../audio-files\Actor_01\03-01-01-01-01-01-01.wav
1,Actor_01,3,1,1,1,1,2,0,../audio-files\Actor_01\03-01-01-01-01-02-01.wav
2,Actor_01,3,1,1,1,2,1,0,../audio-files\Actor_01\03-01-01-01-02-01-01.wav
3,Actor_01,3,1,1,1,2,2,0,../audio-files\Actor_01\03-01-01-01-02-02-01.wav
4,Actor_01,3,1,2,1,1,1,0,../audio-files\Actor_01\03-01-02-01-01-01-01.wav


In [8]:
data.to_csv("local_audio_files.csv", index=False)

## Extracting features

In [9]:
# features:
# mfcc - use n_mfcc=13 and take mean along axis 1 [13 features]
# chroma_stft - take mean along axis 1 [12 features]
# librosa.onset.onset_strength(y=y, sr=sr) - take direct mean
# zero_crossing_rate - take direct mean
# spectral_rolloff - direct mean
# librosa.piptrack - returns pitch and magnitude, take direct means of both
# melspectrogram - take direct mean, and mean along axis=1 [1+127 features]
# spectral_contrast - use axis=1 [7 features]
# tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0) [6 features]
# rms - take direct mean
# spectral_centroid - take direct mean
# spectral_bandwidth - take direct mean


def get_feature_vector(y, sr):
    feature_vector = []
    
    # multi-dim features
    feature_vector.extend(np.mean(feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1))
    feature_vector.extend(np.mean(feature.chroma_stft(y=y, sr=sr), axis=1))
    feature_vector.extend(np.mean(feature.spectral_contrast(y=y, sr=sr), axis=1))
    feature_vector.extend(np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr),axis=1))
    feature_vector.extend(np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1))
    
    # single-dim features with special requirements
    feature_vector.append(np.mean(feature.rms(y=y)))
    feature_vector.append(np.mean(feature.zero_crossing_rate(y=y)))
    feature_vector.extend([np.mean(x) for x in librosa.piptrack(y=y, sr=sr)])
    
    # single-dim features
    feat_list = [
        librosa.onset.onset_strength,
        feature.spectral_rolloff,
        feature.melspectrogram,
        feature.spectral_centroid,
        feature.spectral_bandwidth
    ]
    
    for temp_func in feat_list:
        feature_vector.append(np.mean(temp_func(y=y, sr=sr)))
    
    return feature_vector

In [10]:
# extracting audio features, could take a long time to run
audio_features = []
for i in range(data.shape[0]):
   y , sr = librosa.load(data.iloc[i]['Audio_file'],sr=None)
   feature_vector = get_feature_vector(y, sr)
   audio_features.append(feature_vector) 

In [11]:
data_features = data.copy()

In [12]:
data_features = data_features.drop("Audio_file",axis=1)
data_features.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender
0,Actor_01,3,1,1,1,1,1,0
1,Actor_01,3,1,1,1,1,2,0
2,Actor_01,3,1,1,1,2,1,0
3,Actor_01,3,1,1,1,2,2,0
4,Actor_01,3,1,2,1,1,1,0


In [13]:
data_features['librosa'] = audio_features

## Normalizing with min/max scaling

In [14]:
features = pd.DataFrame(data_features['librosa'].values.tolist())
features.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,-726.217224,68.541420,3.293397,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,5.362133e-09,0.002120,0.050476,37.457439,0.002697,0.891978,13286.038306,0.002952,7416.379545,5551.324979
1,-719.127808,70.202240,1.169071,13.123216,7.837617,14.411950,-4.110705,4.469619,-3.538730,-3.657982,...,5.566884e-09,0.002258,0.052904,40.032833,0.002662,0.992758,13191.718251,0.003416,7135.753114,5653.712371
2,-714.994934,69.690376,3.925557,11.925324,6.423343,11.014113,-2.874456,4.514386,-4.470305,-2.665093,...,5.541867e-09,0.002707,0.046627,37.690022,0.003359,0.860653,13280.282980,0.004721,7240.619346,5640.892215
3,-710.959839,67.579193,5.783356,13.227695,6.194669,12.640195,-1.662046,5.663977,-4.953693,-3.484669,...,5.706893e-09,0.002521,0.053835,39.524185,0.003164,0.918893,13273.018037,0.004341,7009.490125,5802.602446
4,-759.917847,75.788948,6.028997,14.562723,6.459432,14.636641,-2.999552,4.625813,-5.195350,-0.702961,...,5.490418e-09,0.001579,0.045929,38.651924,0.001842,0.988106,12649.614081,0.001624,6997.114097,5518.781643


In [15]:
features.shape

(1440, 175)

In [16]:
labels = data['Emotion']
actor_labels = pd.Series([int(x[-2:]) for x in data['Actor']], name='Actor')

In [17]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

In [18]:
minMax=features.apply(minMax)
minMax

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
min,-873.242310,18.962200,-51.987183,-17.522383,-22.980276,-11.918348,-29.965725,-16.973310,-21.486086,-21.428455,...,1.692426e-09,0.000328,0.027717,2.918853,0.000270,0.774896,4998.104889,0.000077,2604.486405,2753.455274
max,-333.377991,115.150726,22.751472,36.544689,21.137091,28.005917,10.136993,16.415524,6.143632,7.424147,...,1.918034e-07,0.088504,0.168519,56.945759,0.118322,1.711348,14629.079312,4.081514,7654.876393,6367.981144


In [19]:
minMax.to_pickle('Speaker_Classification_data/minMax.df')

In [20]:
features_normalised = features.copy()
for i in range(175):
    features_normalised[i] = features_normalised.apply(lambda x: (x[i]-minMax[i]['min'])/(minMax[i]['max']-minMax[i]['min']), axis=1)
features_normalised.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,0.272337,0.515438,0.739652,0.549830,0.645790,0.640857,0.672820,0.601139,0.657816,0.688467,...,0.019303,0.020323,0.161635,0.639285,0.020555,0.125028,0.860550,0.000705,0.952777,0.774063
1,0.285469,0.532704,0.711228,0.566807,0.698543,0.659506,0.644720,0.642219,0.649567,0.615905,...,0.020380,0.021892,0.178884,0.686954,0.020258,0.232647,0.850756,0.000818,0.897211,0.802389
2,0.293124,0.527383,0.748110,0.544651,0.666486,0.574399,0.675547,0.643559,0.615851,0.650318,...,0.020248,0.026986,0.134300,0.643590,0.026162,0.091577,0.859952,0.001138,0.917975,0.798843
3,0.300599,0.505434,0.772967,0.568739,0.661303,0.615128,0.705780,0.677990,0.598355,0.621912,...,0.021116,0.024872,0.185497,0.677539,0.024511,0.153769,0.859198,0.001045,0.872211,0.843582
4,0.209913,0.590785,0.776254,0.593432,0.667304,0.665134,0.672428,0.646897,0.589609,0.718323,...,0.019978,0.014187,0.129345,0.661394,0.013311,0.227678,0.794469,0.000379,0.869760,0.765059


In [21]:
features_normalised.describe()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
count,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,...,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000
mean,0.474382,0.489855,0.630520,0.512750,0.515861,0.505145,0.538223,0.534816,0.517489,0.548600,...,0.037734,0.110328,0.294735,0.433747,0.100521,0.359410,0.606791,0.033798,0.585344,0.636510
std,0.190016,0.154011,0.157733,0.150477,0.155717,0.179926,0.151327,0.156265,0.179965,0.150803,...,0.037753,0.139412,0.129216,0.219628,0.131401,0.147114,0.156110,0.092327,0.160471,0.157363
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.332606,0.382860,0.532447,0.414962,0.410259,0.370438,0.438937,0.428237,0.385715,0.440547,...,0.025754,0.029084,0.208038,0.231471,0.026313,0.253335,0.497096,0.001074,0.468672,0.529677
50%,0.463760,0.492196,0.640920,0.514474,0.512452,0.512923,0.524175,0.530196,0.497810,0.547954,...,0.032278,0.060842,0.274771,0.467660,0.055306,0.340093,0.626891,0.004247,0.588419,0.661389
75%,0.595785,0.599308,0.743468,0.602627,0.633465,0.640325,0.642005,0.636022,0.655681,0.662356,...,0.039237,0.129380,0.367831,0.613154,0.115368,0.438346,0.725689,0.017331,0.700856,0.752049
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Saving data for SER model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features_normalised, labels, test_size=0.30, random_state=42)

In [23]:
X_train.to_csv('SER_data/X_train.csv', index=False)
X_test.to_csv('SER_data/X_test.csv', index=False)
y_train.to_csv('SER_data/y_train.csv', index=False)
y_test.to_csv('SER_data/y_test.csv', index=False)

## Saving data for Siamese model

In [24]:
# generating all possible pairs, will take a long time to run
total_pairs = []
for i in range(len(features_normalised)-1):
    for j in range(i+1, len(features_normalised)):
        f1 = features_normalised.loc[i, :].values.tolist()
        f2 = features_normalised.loc[j, :].values.tolist()
        if actor_labels[i] == actor_labels[j]:
            l = 1
        else:
            l = 0
        total_pairs.append([f1, f2, l])
pairs_df = pd.DataFrame(total_pairs, columns=['inputA', 'inputB', 'label'])

In [25]:
pairs_df.head()

,inputA,inputB,label
0,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.28546895316601656, 0.5327042826670929, 0.71...",1
1,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.2931243461802796, 0.5273828194015168, 0.748...",1
2,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.3005986227234242, 0.5054344306390837, 0.772...",1
3,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.20991285946164542, 0.5907850984592853, 0.77...",1
4,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.2560564070403884, 0.6251354289835376, 0.804...",1


In [26]:
matched_df = pairs_df[pairs_df['label']==1].copy()
mismatched_df = pairs_df[pairs_df['label']==0].copy()

In [27]:
# shuffling
matched_df = matched_df.sample(frac=1).reset_index(drop=True)
mismatched_df = mismatched_df.sample(frac=1).reset_index(drop=True)

In [28]:
# 50:50 downsampling
downsampled_df = pd.concat([matched_df, mismatched_df[:len(matched_df)].copy()], ignore_index=True)

In [29]:
downsampled_df.describe()

,label
count,84960.000000
mean,0.500000
std,0.500003
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [30]:
# shuffling and taking train-test split

downsampled_df = downsampled_df.sample(frac=1).reset_index(drop=True)
train_size = int(0.7*len(downsampled_df))
train_df = downsampled_df[:train_size].copy()
test_df = downsampled_df[train_size+1:].copy()

In [31]:
test_df.reset_index(drop=True, inplace=True)

In [32]:
train_df.head()

,inputA,inputB,label
0,"[0.6259576307297445, 0.40422018505075, 0.33808...","[0.8785945881900982, 0.04003334094116476, 0.18...",1
1,"[0.512732250501038, 0.41837384487043455, 0.459...","[0.5508735478037253, 0.35679966623366144, 0.40...",1
2,"[0.5705212932258709, 0.40447019253630095, 0.57...","[0.5002341964855943, 0.44852124181224495, 0.70...",1
3,"[0.6716892570459904, 0.2568239174356624, 0.465...","[0.3180312526532941, 0.5282743435568443, 0.729...",0
4,"[0.4092282572898532, 0.6629626243964712, 0.771...","[0.4472490363914092, 0.7145415799053311, 0.654...",1


In [33]:
test_df.head()

,inputA,inputB,label
0,"[0.5142333013044343, 1.0, 0.528514533012731, 0...","[0.5838474859123154, 0.737866278911825, 0.6344...",1
1,"[0.9184059557245976, 0.2592263926049499, 0.368...","[0.439322194783416, 0.6883048330484197, 0.7197...",1
2,"[0.20991285946164542, 0.5907850984592853, 0.77...","[0.4313755682924979, 0.4552303973251103, 0.689...",0
3,"[0.38706602283265934, 0.3531728127170686, 0.76...","[0.464291056204104, 0.307338873764661, 0.57590...",1
4,"[0.5765927650414007, 0.2964150457391947, 0.518...","[0.5946977418212969, 0.2639801834168179, 0.549...",1


In [34]:
train_df.to_pickle('Speaker_Classification_data/train.df')
test_df.to_pickle('Speaker_Classification_data/test.df')